# Streaming Last Agent Response

If you want to attach a callback on the last agent response, you can use the callback ``StreamingLastResponseCallbackHandler``.
For this, the underlying LLM has to support streaming as well.

In [29]:
from langchain.agents import load_tools, initialize_agent, AgentType
from langchain.llms import OpenAI
from langchain.callbacks import StreamingLastResponseCallbackHandler

llm = OpenAI(temperature=0, streaming=True)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=False
)

Now you can initialize the callback handler by using ``StreamingLastResponseCallbackHandler.from_agent_type(agent)``

In [30]:
stream = StreamingLastResponseCallbackHandler.from_agent_type(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

## 1. Using callback function decorator

You can attach a callback function by using the ``on_last_response_new_token()`` decorator.

In [31]:
@stream.on_last_response_new_token()
def on_new_token(token: str):
    if token is StopIteration:
        print("\n[Done]")
        return
    else:
        print(f"Next token: '{token}'")

Now run it with ``agent.run()`` and ``verbose=False``, you can see the callback function is called when the last agent response is received.

In [32]:
agent.run(
    "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
    callbacks=[stream],
)

Next token: ' Gig'
Next token: 'i'
Next token: ' Had'
Next token: 'id'
Next token: ''s'
Next token: ' age'
Next token: ' raised'
Next token: ' to'
Next token: ' the'
Next token: ' 0'
Next token: '.'
Next token: '43'
Next token: ' power'
Next token: ' is'
Next token: ' 4'
Next token: '.'
Next token: '19'
Next token: '06'
Next token: '168'
Next token: '36'
Next token: '1987'
Next token: '195'
Next token: '.'
Next token: ''

[Done]


"Gigi Hadid's age raised to the 0.43 power is 4.1906168361987195."

## 2. Using for-loop

We can create a separate thread to run the agent, and use a for-loop to get the last agent response.

In [ ]:
import threading

stream = StreamingLastResponseCallbackHandler.from_agent_type(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)


def _run():
    agent.run(
        "Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?",
        callbacks=[stream],
    )


threading.Thread(target=_run).start()

for token in stream:
    print(token, end="", flush=True)

 Gigi Hadid's age raised to the 0.43 power is 4.1906168361987195.

## 3. Post process on-the-fly

You can also post process on-the-fly by using ``postprocess`` decorator.

In [23]:
from typing import List
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")

stream = StreamingLastResponseCallbackHandler.from_agent_type(
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)


@stream.postprocess(sliding_window_step=1, window_size=3)
def postprocess_func(tokens: List[str]) -> List[str]:
    sentence = "".join(tokens).replace("Python", "LangChain")
    out_tokens = [
        enc.decode([t]) for t in enc.encode(sentence)
    ]  # postprocess output can have different size!
    return out_tokens


def _run():
    agent.run("Is python good?", callbacks=[stream])


threading.Thread(target=_run).start()

for token in stream:
    print(token, end="", flush=True)

 LangChain is generally considered to be a good language for programming, with many advantages.